# Milestone 2 : Project proposal and initial analyses 🍻

<hr style="clear:both">
This notebook was made for the ADA course at EPFL (CS-401). 

Group : BeerADAventure46

**Authors:** \
[Agatha Hunter](https://people.epfl.ch/agatha.hunter)\
[Anne-Valérie Preto](https://people.epfl.ch/anne-valerie.preto)\
[Tristan Carruzzo](https://people.epfl.ch/tristan.carruzzo)\
[Thamin Maurer](https://people.epfl.ch/thamin.maurer)\
[Victor Dubien](https://people.epfl.ch/victor.dubien)


**Supervisor:**\
[Beatriz Borges](https://people.epfl.ch/beatriz.borges)
<hr style="clear:both">



<blockquote>

## Project proposal  📌

**Influence of the serving style on the tasting profile**:

Goal of the project: Use the textual reviews to find for each beer how it was served (either bottle, can or draft) and find how it influences the tasting profile.

Different serving styles may be adopted in different regions or for different types of beer.  The influence of the serving style could influence not only the appearance, but also the aroma, the palate or the taste.
</blockquote>

### TODO LIST FOR THE GROUP:

<input type="checkbox" checked > New architecture of file

<input type="checkbox" checked > Create functions to import and convert initial files

<input type="checkbox" checked > More comments on the dataset -> distribution, viz

<input type="checkbox" checked > For any filtering: how, why ?

<input type="checkbox"> Choose a NLP method -> comment as much as possible, some trials 

<input type="checkbox" checked > E-mail the proposition to Beatriz

### Import libraries

In [1]:
import io
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from langdetect import detect
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
import time

from textstat import flesch_reading_ease, flesch_kincaid_grade, gunning_fog, smog_index, automated_readability_index, coleman_liau_index, linsear_write_formula, dale_chall_readability_score, text_standard
from utils import *
pd.set_option('display.max_columns', None)

dataset_path = './data/BeerAdvocate/'

## Datasets 📂
### Import dataset

In [ ]:
# Firt time imports : 
# Convert txt to csv (function in the utils.py file)

#ratings = convert_txt_to_csv(input_file='ratings.txt', export=True, file_name='ratings')
#reviews = convert_txt_to_csv(input_file='reviews.txt', export=True, file_name='reviews')

In [ ]:
# Files already converted, so we can load them directly

reviews = pd.read_csv(dataset_path + 'reviews.csv')
beers = pd.read_csv(dataset_path + 'beers.csv')
breweries = pd.read_csv(dataset_path + 'breweries.csv')
users = pd.read_csv(dataset_path + 'users.csv')

### Initial datasets analysis 🔎

#### Beers dataset 🍺

In [ ]:
display(beers.sample(3))
print('Shape of beers : ', beers.shape)

In [ ]:
# print the number of beers id and beer name
print('Number of different beers id:', len(beers.beer_id.unique()))
print('Number of different beers name:', len(beers.beer_name.unique()))

<blockquote>

**Initial comments about the dataset**

-Some beers have the same name! In order to avoid confusion, we will drop the column beer_name (and brewery_name) from the dataset.

-As we will not use the RateBeer dataset, we can also drop the column nbr_matched_valid_ratings and avg_matched_valid_ratings.  

-The column bros_score only reflect the ratings from the two creators of the website, so we can also drop it.  

-ba_score represents the percentage (rounded to the tenth for depth) of raters who gave the beer a 3.75 or higher as long as the beers as at least 10 ratings.

</blockquote>

In [ ]:
# show the percentage of missing values per columm
plt.figure()
(beers.isna().sum() / len(beers) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

#print how many values are missing in each column in %
print('How many values missing per column in %:')
display(beers.isna().sum()/ len(beers) * 100)

<blockquote>

**More comments about the dataset**

We see that more than 80% of the beers do not have a z-score value. We can drop this column alongside avg_computed and compute them manually later if needed from the reviews dataset. 

80% of the beers have no ba_score value, so we can also drop it and compute it later if needed.

The other columns with NaN such as avg_matched_valid_ratings, avg_computed or bros_score are not usefull in our analysis since we only look at the BeerAdvocate reviews.

However, the abv, which represents the alcohol percentage in a beer, is missing for approximately 12%.

</blockquote>

In [ ]:
beers_filtered = beers.drop(columns=['beer_name', 
                            'brewery_name',
                            'bros_score',
                            'ba_score',
                            'nbr_matched_valid_ratings',
                            'avg_matched_valid_ratings',
                            'zscore',
                            'avg_computed']).copy(deep=True)

beers_filtered.rename(columns={'nbr_ratings': 'beer_nbr_ratings',
                               'nbr_reviews': 'beer_nbr_reviews'}, inplace=True)

In [ ]:
beers_filtered.sample(3)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
beers_filtered['style'].value_counts(normalize=False).head(10).plot(kind='barh', title='10 most common beer styles')
plt.xlabel('Number of beers')
plt.subplot(1, 2, 2)
beers_filtered['beer_nbr_reviews'].hist(bins=50,log=True)
plt.title('Number of reviews per beer')
plt.ylabel('Count')
plt.xlabel('Number of reviews')
plt.tight_layout()
plt.show()

In [ ]:
# show boxplot of abv for each beer style
plt.figure(figsize=(15, 15))
sns.boxplot(y='style', x='abv', data=beers_filtered, fliersize=3)
plt.yticks(fontsize=8)
plt.title('Boxplot of abv for each beer style')
plt.show()

In [ ]:
# group the beers by style and compute the median of abv for each style
median_abv_by_style = beers_filtered.groupby('style')['abv'].median()
display(median_abv_by_style.sample(3))

# fill the missing abv values with the median of abv for the style of each beer
beers_filtered['abv'] = beers_filtered.apply(lambda x: median_abv_by_style[x['style']] if pd.isna(x['abv']) else x['abv'], axis=1)

In [ ]:
beers_filtered.describe()

<blockquote>

We are left with a dataset containing 280823 beers. We are only missing some values for avg as some beers do not have any ratings/reviews. They will be removed later when merging with the reviews dataset anyway.

</blockquote>

#### Breweries dataset 🏭

In [ ]:
display(breweries.sample(3))
print('Shape of breweries : ', breweries.shape)

In [ ]:
# show the percentage of missing values per columm
plt.figure()
(breweries.isna().sum() / len(breweries) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

<blockquote>

This dataset has no missing values, we will simply rename the columns and drop the brewery name.

</blockquote>

In [ ]:
breweries_filtered = breweries.drop(columns=['name']).copy(deep=True)
breweries_filtered = breweries.copy(deep=True)

breweries_filtered.rename(columns={'id':'brewery_id',
                                   'location':'brewery_location',
                                   'nbr_beers':'brewery_nbr_beers'}, inplace=True)

In [ ]:
breweries_filtered.sample(3)

In [ ]:
# Some breweries have weird location values, should find a way to clean them
print('Number of breweries with a weird location value:', len(breweries_filtered[breweries_filtered['brewery_location'].str.contains('<', na=False)]))

# should manully add the us state of the 35 breweries because it corresponds to to 15'319 reviews.

# for loc in breweries_filtered[breweries_filtered['brewery_location'].str.contains('<', na=False)]['brewery_location'].unique():
#     print(loc) # print the strange values

In [ ]:
# manually adding the location for those 35 breweries

#obtain the brewery_id of the 35 breweries
strange_id = breweries_filtered[breweries_filtered['brewery_location'].str.contains('<', na=False)]['brewery_id'].unique()
strange_id

# create a dic with the brewery_id as key and unknwon as value
dic = {}
for i in strange_id:
    dic[i] = 'unknown'

In [ ]:
# create a function that takes as input the brewery_id and a location and change the location of the brewery with the given id in the dictionary dic 
def change_location(brewery_id, us_state):
    location = 'United States, ' + us_state
    dic[brewery_id] = location

In [ ]:
# take the first element of dic that has value unknown and print its location
for key, value in dic.items():
    if value == 'unknown':
        print('Brewery id:', key)
        # print('Brewery name:', breweries_filtered[breweries_filtered['brewery_id'] == key]['name'].values[0])
        print('Strange location:',breweries_filtered[breweries_filtered['brewery_id'] == key]['brewery_location'].values[0])        
        break

In [ ]:
#Changing the location in dic

change_location(brewery_id=2434,us_state='New Mexico')
change_location(brewery_id=2730,us_state='Wisconsin')
change_location(brewery_id=1551,us_state='California')
change_location(brewery_id=1552,us_state='Michigan')
change_location(brewery_id=2710,us_state='California')
change_location(brewery_id=3681,us_state='Ohio')
change_location(brewery_id=2413,us_state='California')
change_location(brewery_id=1793,us_state='Virginia')
change_location(brewery_id=6045,us_state='New Jersey')
change_location(brewery_id=2776,us_state='New York')
change_location(brewery_id=1751,us_state='Texas')
change_location(brewery_id=1554,us_state='Georgia')
change_location(brewery_id=1961,us_state='Maine')
change_location(brewery_id=4927,us_state='South Carolina')
change_location(brewery_id=23973,us_state='Maine')
change_location(brewery_id=2512,us_state='California')
change_location(brewery_id=6416,us_state='New Jersey')
change_location(brewery_id=2104,us_state='Oregon')
change_location(brewery_id=3079,us_state='Washington')
change_location(brewery_id=2410,us_state='Michigan')
change_location(brewery_id=1931,us_state='Ohio')
change_location(brewery_id=1553,us_state='Maine')
change_location(brewery_id=1550,us_state='Colorado')
change_location(brewery_id=32764,us_state='Illinois')
change_location(brewery_id=8451,us_state='Wyoming')
change_location(brewery_id=5114,us_state='California')
change_location(brewery_id=1802,us_state='North Carolina')
change_location(brewery_id=1819,us_state='Minnesota')
change_location(brewery_id=200,us_state='California')
change_location(brewery_id=918,us_state='Florida')
change_location(brewery_id=4343,us_state='California')
change_location(brewery_id=9765,us_state='New York')
change_location(brewery_id=1927,us_state='Colorado')
change_location(brewery_id=28908,us_state='Utah')
change_location(brewery_id=70,us_state='Oregon')
change_location(brewery_id=1567,us_state='Hawaii')

In [ ]:
# change the location of the breweries in the breweries_filtered dataframe
for key, value in dic.items():
    breweries_filtered.loc[breweries_filtered['brewery_id'] == key, 'brewery_location'] = value

In [ ]:
# how many different locations are there that contain United States
print('Number of different locations that contain United States:', len(breweries_filtered[breweries_filtered['brewery_location'].str.contains('United States')]['brewery_location'].unique()))

In [ ]:
# plot the number of breweries per state as barh
plt.figure(figsize=(15, 15))
breweries_filtered[breweries_filtered['brewery_location'].str.contains('United States')]['brewery_location'].value_counts().plot(kind='barh')
plt.title('Number of breweries per state')
plt.xlabel('Number of breweries')
plt.ylabel('State')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
breweries_filtered['brewery_nbr_beers'].hist(bins=50, log=True)
plt.title('Number of beers per brewery')
plt.ylabel('Count')
plt.xlabel('Number of beers')
plt.subplot(1, 2, 2)
breweries_filtered['brewery_location'].value_counts(normalize=False).head(10).plot(kind='barh', title='10 most common brewery locations')
plt.xlabel('Number of breweries')
plt.tight_layout()
plt.show()

#### Users dataset 👤

In [ ]:
display(users.sample(3))
print('Shape of users : ', users.shape)

In [ ]:
plt.figure()
(users.isna().sum() / len(users) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

<blockquote>

We are only missing information about when the user joined the website and locations. (and 1 username, which we drop anyway)

</blockquote>

In [ ]:
users_filtered = users.drop(columns=['user_name']).copy(deep=True)

users_filtered.rename(columns={'nbr_ratings':'user_nbr_ratings',
                               'nbr_reviews':'user_nbr_reviews',
                               'joined':'user_joined',
                               'location':'user_location'}, inplace=True)

users_filtered.sample(3)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
users_filtered['user_nbr_reviews'].hist(bins=50, log=True)
plt.title('Number of reviews per users')
plt.ylabel('Count')
plt.xlabel('Number of reviews')
plt.subplot(1, 2, 2)
users_filtered['user_location'].value_counts(normalize=False).head(10).plot(kind='barh', title='10 most common user locations')
plt.xlabel('Number of users')
plt.tight_layout()
plt.show()

<blockquote>

We observe that many users have no reviews at all, they will be removed from the dataset when merging. Also, the 10 most common user location are all in the US.

</blockquote>

#### Reviews dataset 📝

In [ ]:
display(reviews.sample(3))
print('Shape of reviews : ', reviews.shape)

In [ ]:
plt.figure()
(reviews.isna().sum() / len(reviews) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

In [ ]:
# making sure that all the NaN are in the same rows

# print the number of rows that contains a NaN value for each column ['overall', 'taste', 'palate', 'aroma', 'appearance']
print('Number of rows with NaN values for each column :')
for col in ['overall', 'taste', 'palate', 'aroma', 'appearance']:
    print(col, ':', len(reviews[reviews[col].isna()]))
    
missing_values = reviews[['overall', 'taste', 'palate', 'aroma', 'appearance']].isna()
missing_values['all_missing'] = missing_values.all(axis=1)
print(missing_values['all_missing'].value_counts())

<blockquote>

All the missing values are in the same rows, and they represent less than 1.5% of the dataset. We can drop them.

</blockquote>

In [ ]:
# remove the columns that are not useful for our analysis
reviews_filtered = reviews.drop(columns=['user_name', 'beer_name', 'brewery_name']).copy(deep=True)

# remove the column abv because we have the information in the beers dataframe
reviews_filtered.drop(columns=['abv'], inplace=True)

#remove rows in reviews_filtered that have all_missing True in missing_values
reviews_filtered = reviews_filtered[~missing_values['all_missing']].copy(deep=True)

display(reviews_filtered.sample(3))
print('Shape of reviews_filtered : ', reviews_filtered.shape)

In [ ]:
# count the number of NaN in columns ['overall', 'taste', 'palate', 'aroma', 'appearance']
print('Number of NaN values for each column :')
for col in ['overall', 'taste', 'palate', 'aroma', 'appearance']:
    print(col, ':', len(reviews_filtered[reviews_filtered[col].isna()]))


In [ ]:
# show boxplot for each rating column, appearance, aroma, palate, taste and overall

plt.figure(figsize=(8, 5))
sns.boxplot(data=reviews_filtered[['appearance', 'aroma', 'palate', 'taste', 'overall']])
plt.title('Beer Ratings by Aspect')
plt.xlabel('Aspect')
plt.ylabel('Rating')
plt.show()

### Merge datasets 🔗

In [ ]:
# Merge reviews with users and beers and breweries
df_merged = reviews_filtered.merge(beers_filtered, how='left', on=['beer_id', 'brewery_id', 'style'])
df_merged = df_merged.merge(users_filtered, how='left', on='user_id')
df_merged = df_merged.merge(breweries_filtered, how='left', on='brewery_id')

display(df_merged.sample(1))
print('Shape of df_merged : ', df_merged.shape)

### Filter datasets ✂️

<div class="alert alert-block alert-danger">
<b>Danger:</b> Comment EVERY filtering
</div>

In [ ]:
# show the percentage of missing values per columm
plt.figure()
(df_merged.isna().sum() / len(df_merged) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

In [ ]:
# how many reviews have strange brewery location ?
print('Number of reviews with a weird location value:', len(df_merged[df_merged['brewery_location'].str.contains('<', na=False)]))

In [ ]:
# how many reviews have no abv value ?
print('Number of reviews with no abv value:', len(df_merged[df_merged['abv'].isna()]))

In [ ]:
# We can replace missing user_joined values with the date of the first review of the user
df_merged['first_review'] = df_merged.groupby('user_id')['date'].transform('min')
df_merged['user_joined'] = df_merged['user_joined'].fillna(df_merged['first_review'])
df_merged.drop(columns=['first_review'], inplace=True)

In [ ]:
# What is the percentage of reviews with at least one NaN value ?
print('Percentage of reviews with at least one NaN value :', len(df_merged[df_merged.isna().any(axis=1)]) / len(df_merged) * 100)
print('This corresponds to', len(df_merged[df_merged.isna().any(axis=1)]), 'reviews')

In [ ]:
# show the percentage of missing values per columm
plt.figure()
(df_merged.isna().sum() / len(df_merged) * 100).plot(kind='barh', title='Percentage of missing values per column')
plt.xlabel('Percentage of missing values')
plt.show()

In [ ]:
# Save to csv
df_merged.to_csv(dataset_path + 'BeerAdvocateMerged.csv', index=False)

## Data enrichment 🍺

In [2]:
df = pd.read_csv(dataset_path + 'BeerAdvocateMerged.csv')

In [3]:
display(df.sample(3))
print('Shape of df : ', df.shape)

,beer_id,brewery_id,style,date,user_id,appearance,aroma,palate,taste,overall,rating,text,beer_nbr_ratings,beer_nbr_reviews,avg,abv,user_nbr_ratings,user_nbr_reviews,user_joined,user_location,brewery_location,name,brewery_nbr_beers
1180153,6076,651,American IPA,1290078000,garuda.417017,5.0,5.0,4.5,4.0,5.0,4.55,A - Hazed deep golden with a towering white he...,5272,1288,4.17,7.5,1155,471,1.263812e+09,"United States, Pennsylvania","United States, New York",Ithaca Beer Company,193
752470,48508,13839,Euro Dark Lager,1243591200,ffejherb.103416,4.0,4.0,4.0,4.0,4.0,4.00,22oz. bomber into shaker pint on 5/27/09. Than...,365,183,3.77,6.5,1661,1647,1.161166e+09,"United States, Pennsylvania","United States, California",Port Brewing,68
1757181,65987,23145,Winter Warmer,1333447200,georgiabeer.87185,3.5,4.0,3.5,3.0,3.5,3.42,Overall I was a little disappointed in this be...,79,28,4.04,8.8,3194,2320,1.152094e+09,"United States, Georgia","United States, North Carolina",Fullsteam Brewery,96


Shape of df :  (2557577, 23)


### Serving type (naive approach) 🍾🥤🚰

>Since our goal is to look at the influence of the serving style, we need to know how each beer was served. 
For now, we use a naive approach. We only look at whether a word specific to as service style has been used. 

In [ ]:
# Create 3 different columns according to the text review, "bottle", "can", "draft"

bottle = ['bottle', 'bottled', 'bottles']
can = [' can ', 'canned', ' cans ']
draft = ['draft', 'draught', 'tap', 'taps']
# Create different columns if review text contains the word
df['bottle'] = df['text'].astype(str).apply(lambda x: any(ele in x.lower() for ele in bottle))
df['can'] = df['text'].astype(str).apply(lambda x: any(ele in x.lower() for ele in can))
df['draft'] = df['text'].astype(str).apply(lambda x: any(ele in x.lower() for ele in draft))

In [ ]:
# remove every row where columns bottle, can and draft are all False
df = df[df[['bottle', 'can', 'draft']].any(axis=1)]
# display(df.sample(3))
# print('We have {} valid reviews'.format(df.shape[0]))

# remove every row with more than 1 true
df = df[df[['bottle', 'can', 'draft']].sum(axis=1) == 1]
# display(df.sample(3))
# print('We have {} valid reviews'.format(df.shape[0]))

#create a new column with the serving type
df['serving_type'] = df[['bottle', 'can', 'draft']].idxmax(axis=1)

# drop the columns bottle, can and draft
df.drop(['bottle', 'can', 'draft'], axis=1, inplace=True)

display(df.sample(3))

# print the number of reviews for each serving type
df['serving_type'].value_counts(normalize=False)

In [ ]:
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def calculate_similarity(review, word_bottle, word_can,  word_draft ):
    # Read the list of words from the text file
    with open(word_can, 'r') as file:
        word_can = file.read().splitlines()

    with open(word_bottle, 'r') as file:
        word_bottle = file.read().splitlines()

    with open(word_draft, 'r') as file:
        word_draft = file.read().splitlines()

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')

    # Transform the review and words list using TF-IDF
    tfidf_matrix = vectorizer.fit_transform([review])

    can_vector = vectorizer.transform([" ".join(word_can)])
    bottle_vector = vectorizer.transform([" ".join(word_bottle)])
    draft_vector = vectorizer.transform([" ".join(word_draft)])

    #Calculate the cosine similarity
    cosine_sim_can = cosine_similarity(tfidf_matrix, can_vector)
    cosine_sim_bottle = cosine_similarity(tfidf_matrix, bottle_vector)
    cosine_sim_draft = cosine_similarity(tfidf_matrix, draft_vector)

    return cosine_sim_bottle, cosine_sim_can, cosine_sim_draft

# Example usage
text = df.sample(1)['text'].values[0].lower()
print(text, '\n')
word_bottle = "data/bottle.txt"
word_can = "data/can.txt"
word_draft = "data/draft.txt"

cosine_sim_bottle, cosine_sim_can, cosine_sim_draft = calculate_similarity(text, word_bottle, word_can,  word_draft )
print("Cosine similarity for bottle:", cosine_sim_bottle[0][0])
print("Cosine similarity for can:", cosine_sim_can[0][0])
print("Cosine similarity for draft:", cosine_sim_draft[0][0])

<blockquote>
It seems to work for most reviews. It might be interesting to save this feature in the reviews df and compare it to the naive method. 

For a few example, two similarity score seems to come close. <br>
<hr>

*poured from a 500 ml bottle into a nonic pint. excellent label of a beer-swilling elephant riding a steam engine. a hilariously sick tribute to local martyr &quot;jumbo&quot;, p.t. barnum's famous elephant, who was crushed to death by a train in st. thomas in the 1880's.deep, clear gold tending toward amber, with a tight creamy head that eventually settles down to a soft, white collar. good lacing.smell is some spicy, piney c-hop; sweet, toffee-like malt; and on the fainter side.taste follows smell. the principal malt character is toffee-like sweetness. intensely sweet upfront, followed by a prickly bitterness that smooths out on the way down and rears its head in the lingering, bittersweet aftertaste. it actually leans toward sweet to the point of underbalance, if only slightly. the faintest hints of staleness/infection (one of the flavors i'm actually quite good at picking out), but nothing serious.mouthfeel is moderate/full, lighter carbonation, refreshing.only okay drinkability. i can't put my finger on it, but with every sip i find myself wanting less. it's definitely leans toward the sticky-sweet, so maybe that's why.one of the better beers from ironspike. good to try, but i wouldn't go out of my way for it.*


It gave the following similarity score: 

can : 0.0

bottle : 0.0686369473085648 (because of bottle)

draft : 0.07000595987603604 (because of pint)

The label should be "bottle", but it's still interesting that this user drank it from a glass. The difference in those moments might be interesting to look at


</blockquote>

<div class="alert alert-block alert-warning">
<b>Interesting observation:</b> Apparently, most of the time the users who had it on can or bottle serve it nonetheless on a glass -> Probably interesting to look at later.
</div>


In [ ]:
# create 3 new columns (each service type) with the similarity score for each review
# takes really long, more than 30 minutes
df[['similarity_bottle', 'similarity_can', 'similarity_draft']] = df['text'].apply(lambda x: pd.Series(calculate_similarity(x, word_bottle, word_can, word_draft)))

### Sentiment analysis 📝

In [ ]:
# Load positive and negative word lists
with open("data/positive.txt", "r") as f:
    positive_words = f.read().splitlines()

with open("data/negative.txt", "r") as f:
    negative_words = f.read().splitlines()
    
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
text = df.sample(1)['text'].values[0].lower()
print(text)

### textblob
print('Polarity textblob:',TextBlob(text).sentiment.polarity)
# print('Subjectivity:',TextBlob(text).sentiment.subjectivity)

### TF-IDF
# Fit and transform the review text
tfidf_matrix = vectorizer.fit_transform([text])

# Calculate the cosine similarity between the review and positive/negative word vectors
positive_vector = vectorizer.transform([" ".join(positive_words)])
negative_vector = vectorizer.transform([" ".join(negative_words)])

cosine_sim_pos = cosine_similarity(tfidf_matrix, positive_vector)
cosine_sim_neg = cosine_similarity(tfidf_matrix, negative_vector)

# Calculate the polarity score
polarity_score = cosine_sim_pos - cosine_sim_neg

print("Polarity tf-idf:", polarity_score[0][0])

In [ ]:
# function who does the textblob and tf-idf polarity score for each review

def polarity_score(review, positive_vector, negative_vector):
    # textblob
    polarity_textblob = TextBlob(review).sentiment.polarity
    # tf-idf
    #tfidf_matrix = vectorizer.fit_transform([review])
    #positive_vector = vectorizer.transform([" ".join(positive_words)])
    #negative_vector = vectorizer.transform([" ".join(negative_words)])
    #cosine_sim_pos = cosine_similarity(tfidf_matrix, positive_vector)
    #cosine_sim_neg = cosine_similarity(tfidf_matrix, negative_vector)
    #polarity_tfidf = cosine_sim_pos - cosine_sim_neg
    return polarity_textblob #, polarity_tfidf[0][0]

In [ ]:
# create columns for the textblob and tf-idf polarity score for each review
#took 6min to run
#df[['polarity_textblob', 'polarity_tfidf']] = df['text'].apply(lambda x: pd.Series(polarity_score(x, positive_vector, negative_vector)))
df[['polarity_textblob']] = df['text'].apply(lambda x: pd.Series(polarity_score(x, positive_vector, negative_vector)))

In [ ]:
display(df)

In [ ]:
#export df to csv   
df.to_csv(dataset_path + 'BeerAdvocateMerged2.csv', index=False)

## Data filtering 🧹
Number of valid reviews per beer and users & average rating per beer

### Filtering on amount of reviews

In [ ]:
# compute the number of valid reviews per user
df['user_nbr_reviews_computed'] = df.groupby('user_id')['user_id'].transform('count')

# compute the number of valid reviews per beer
df['beer_nbr_reviews_computed'] = df.groupby('beer_id')['beer_id'].transform('count')

# compute the avg rating per beer
df['beer_avg_computed'] = df.groupby('beer_id')['rating'].transform('mean')

df.sample(3)

In [ ]:
plt.figure(figsize=(15, 5))

# group the reviews by beer_id and show a histogram of the number of reviews computed
plt.subplot(1, 2, 1)
df.groupby('beer_id')['beer_nbr_reviews_computed'].mean().hist(bins=100, log=True)
plt.xlabel('Number of reviews per beer')
plt.ylabel('Count')

# group the reviews by user_id and show a histogram of the number of reviews computed
plt.subplot(1, 2, 2)
df.groupby('user_id')['user_nbr_reviews_computed'].mean().hist(bins=100, log=True)
plt.xlabel('Number of reviews per user')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
#Filtering to remove users and beers with less reviews

min_nbr_reviews_per_user = 0
min_nbr_reviews_per_beer = 20

len0 = df.shape[0]

# remove every row where the number of reviews per user is less than min_nbr_reviews_per_user
df_filter = df[df['user_nbr_reviews_computed'] >= min_nbr_reviews_per_user].copy(deep=True)

# remove every row where the number of reviews per beer is less than min_nbr_reviews_per_beer
df_filter_full = df_filter[df_filter['beer_nbr_reviews_computed'] >= min_nbr_reviews_per_beer].copy(deep=True)

print('We removed {} rows, leaving us with {} %% of the original dataset'.format(len0 - df_filter_full.shape[0], round(df_filter_full.shape[0] / len0 * 100, 2)))
print('We have {} valid reviews'.format(df_filter_full.shape[0]))

# print the number of reviews for each serving type
df_filter_full['serving_type'].value_counts(normalize=False)

### Filter on readability score of the reviews 📝

In [ ]:
df_sample = df_filter_full.sample(n=1000, random_state=46)

In [ ]:
df_sample['readability_score'] = df_sample['text'].apply(lambda x: flesch_reading_ease(x) if isinstance(x, str) else None)
df_sample['readability_score'].hist(bins=100, figsize=(15, 5), log=True)

In [ ]:
# printing some reviews based on readability score

df_sample[df_sample['readability_score'] < 20]['text'].count()

# for row in df_sample[df_sample['readability_score'] > 96]['text']:
#     print(row)
#     print('----------------------')

In [ ]:
# remove reviews with bad readability score



### Extracting the serving mode with NLP

In [4]:
import nltk
import spacy
from nltk import word_tokenize, pos_tag, ne_chunk
nlp = spacy.load("en_core_web_sm")

In [5]:
#250 reviews for function evaluation (manually labelled)
df_eval = df.sample(n=250, random_state=46).copy(deep=True)
df_eval['true_serving_type'] = 'not_set'

In [6]:
# split the dataset in 5 parts
df_eval_tristan = df_eval.iloc[:50].copy(deep=True)
df_eval_av = df_eval.iloc[50:100].copy(deep=True)
df_eval_victor = df_eval.iloc[100:150].copy(deep=True)
df_eval_thamin = df_eval.iloc[150:200].copy(deep=True)
df_eval_agatha = df_eval.iloc[200:].copy(deep=True)

In [7]:
def display_review(df):
    # filter the dataframe to only include rows where 'true_serving_type' is 'unknown'
    df_not_set = df[df['true_serving_type'] == 'not_set']
    if len(df_not_set) == 0:
        print('No more reviews to label')
        return None
    else:
        # print the 'text' column of the first row with unknown serving type
        index = df_not_set.index[0]
        print(f"Review {index}:\n{df_not_set.loc[index, 'text']}")
        
        return index

def update_review_serving_type(df, index):
    # ask the user to input the serving type
    if index is None:
        return df
    else:
        serving_type = input("Enter the serving type (bottle/can/draft/unknown): ")
        if serving_type not in ['bottle', 'can', 'draft', 'unknown']:
            print('Invalid serving type')
            return df
        else:
            # update the 'true_serving_type' column of the review with the given index
            df.loc[index, 'true_serving_type'] = serving_type
            return df

In [9]:
# Manual labelling Tristan
idx = display_review(df_eval_tristan)
df_eval_tristan = update_review_serving_type(df_eval_tristan, idx)

No more reviews to label


In [8]:
# export to save true labels
# df_eval_tristan.to_csv('BeerAdvocateEvalTristan.csv', index=True)
df_eval_tristan = pd.read_csv('BeerAdvocateEvalTristan.csv', index_col=0)

In [10]:
df_eval_tristan['true_serving_type'].value_counts(normalize=False)

true_serving_type
unknown    26
bottle     22
draft       1
can         1
Name: count, dtype: int64

In [ ]:
# Manual labelling AV
idx = display_review(df_eval_av)
df_eval_av = update_review_serving_type(df_eval_av, idx)

In [ ]:
# Manual labelling Victor
idx = display_review(df_eval_victor)
df_eval_av = update_review_serving_type(df_eval_victor, idx)

In [ ]:
# Manual labelling Thamin
idx = display_review(df_eval_thamin)
df_eval_av = update_review_serving_type(df_eval_thamin, idx)

In [ ]:
# Manual labelling Agatha
idx = display_review(df_eval_agatha)
df_eval_av = update_review_serving_type(df_eval_agatha, idx)

In [11]:
# Original functions

def differentiate_can(sentence):
    doc = nlp(sentence)
    verbes_nom=[]
    for token in doc:
        if token.text.lower() == "can":
            if any(t.dep_ == "aux" for t in token.head.children):
                verbes_nom .append("verb")
            else:
                verbes_nom .append("noun") 
    
    return verbes_nom

def remove_conditional(review):
    # Tokenize the review
    doc = nlp(review)

    modified_tokens = []

    for sent in doc.sents:
        remove_sentence = False
        for token in sent:
            if remove_sentence:
                continue
            if token.text.lower() == "would" or token.text.lower() == "could":
                # Identify the subtree dependent on the conditional word
                subtree = [t.text for t in token.subtree]
                # Join the modified tokens to form the modified text
                modified_tokens.extend(subtree[:-1])  # Exclude the conditional word itself
                remove_sentence = True
            else:
                modified_tokens.append(token.text)

    # Join the modified tokens to form the modified text
    modified_text = " ".join(modified_tokens)

    return modified_text

def extract_serving_style(review):
    serving_style = 'unknown'
    # Tokenize the review
    modified_review = remove_conditional(review)
    modified_words = nlp(modified_review)
    bottle = ['bottle', 'bottled', 'bottles', 'growler']
    can = ['can', 'canned', ' cans ']
    draft = ['draft', 'draught', 'tap', 'taps']

    serving_styles = []

    for word in modified_words:
        # If the word is a serving style, add it to the list
        if word.text.lower() in bottle:
            serving_styles.append("bottle")
        if word.text.lower() in can:
            serving_styles.append("can")
        if word.text.lower() in draft:
            serving_styles.append("draft")
    
    #If the list contains "can" we need to check the output of differentiate_can to see if there are several instances of "can". If it is at least one instance of "can" as a noun, we keep "can" in "serving_styles", otherwise we remove it
    if "can" in serving_styles:
        if "noun" in differentiate_can(modified_review):
            serving_styles = serving_styles
        else:
            serving_styles.remove("can")
    

    # If the list contains more than one different serving style, we put "unknown"
    if len(set(serving_styles)) > 1:
        serving_style = "unknown"
    # If the list contains only one serving style, we put this serving style
    elif len(set(serving_styles)) == 1:
        serving_style = serving_styles[0]
    
    return serving_style

# Accuracy function
def compute_accuracy(predictions, true_classes):
    correct_count = 0
    total_count = len(predictions)

    for pred, true_class in zip(predictions, true_classes):
        if pred=='unknwon' and true_class=='unknown':
            correct_count += 1
        elif pred == true_class:
            correct_count += 1

    accuracy = correct_count / total_count if total_count > 0 else 0.0
    return accuracy

In [12]:
# GPT-improved

def gpt_differentiate_can(sentence):
    doc = nlp(sentence)
    verbes_nom = []
    for token in doc:
        if token.text.lower() == "can" and any(t.dep_ == "aux" for t in token.head.children):
            verbes_nom.append("verb")
        elif token.text.lower() == "can":
            verbes_nom.append("noun")

    return verbes_nom

def gpt_remove_conditional(review):
    doc = nlp(review)
    
    modified_tokens = []
    remove_sentence = False

    for sent in doc.sents:
        for token in sent:
            if remove_sentence:
                continue
            if token.text.lower() in {"would", "could"}:
                subtree = [t.text for t in token.subtree]
                modified_tokens.extend(subtree[:-1])
                remove_sentence = True
            else:
                modified_tokens.append(token.text)

    modified_text = " ".join(modified_tokens)

    return modified_text

def gpt_extract_serving_style(review):
    bottle = {'bottle', 'bottled', 'bottles', 'growler'}
    can = {'can', 'canned', 'cans'}
    draft = {'draft', 'draught', 'tap', 'taps'}

    modified_review = gpt_remove_conditional(review)
    modified_words = nlp(modified_review)

    serving_styles = []
    for word in modified_words:
        # If the word is a serving style, add it to the list
        if word.text.lower() in bottle:
            serving_styles.append("bottle")
        if word.text.lower() in can:
            serving_styles.append("can")
        if word.text.lower() in draft:
            serving_styles.append("draft")

    if "can" in serving_styles and "noun" not in gpt_differentiate_can(modified_review):
        serving_styles.remove("can")

    serving_style = serving_styles[0] if len(set(serving_styles)) == 1 else 'unknown'

    return serving_style

In [13]:
# extract serving style for each review
t1 = time.time()
df_eval_tristan['extracted_serving_type'] = df_eval_tristan['text'].apply(lambda x: extract_serving_style(x))
t2 = time.time()
print('Time to run the function on {} reviews: {} seconds'.format(len(df_eval_tristan), round(t2 - t1, 5)))
print('Accuracy:', compute_accuracy(df_eval_tristan['extracted_serving_type'], df_eval_tristan['true_serving_type']))

time_needed = (len(df) / len(df_eval_tristan)) * (t2 - t1)
print('Time needed to run the function on {} reviews: {} hours'.format(len(df), round(time_needed/3600, 1)))

Time to run the function on 50 reviews: 2.75111 seconds
Accuracy: 0.96
Time needed to run the function on 2557577 reviews: 39.1 hours


In [14]:
# extract serving style for each review with gpt functions
t1 = time.time()
df_eval_tristan['gpt_extracted_serving_type'] = df_eval_tristan['text'].apply(lambda x: gpt_extract_serving_style(x))
t2 = time.time()
print('Time to run the function on {} reviews: {} seconds'.format(len(df_eval_tristan), round(t2 - t1, 5)))
print('Accuracy:', compute_accuracy(df_eval_tristan['gpt_extracted_serving_type'], df_eval_tristan['true_serving_type']))

time_needed = (len(df) / len(df_eval_tristan)) * (t2 - t1)
print('Time needed to run the function on {} reviews: {} hours'.format(len(df), round(time_needed/3600, 1)))

Time to run the function on 50 reviews: 2.21757 seconds
Accuracy: 0.96
Time needed to run the function on 2557577 reviews: 31.5 hours


In [15]:
# loop through the mismatched rows
for index, row in df_eval_tristan[df_eval_tristan['extracted_serving_type'] != df_eval_tristan['true_serving_type']].iterrows():
    print(f"Review {index}:\n{row['text']}\nExtracted serving type: {row['extracted_serving_type']}\nTrue serving type: {row['true_serving_type']}")
    print('----------------------')

Review 2148077:
poured from 12 oz. bottledeep reddish mahogany or dark amber honey colored with enough haze to render it opaque. creamy off-white head that left chunky bits of lacing stuck to the inside of the glass all the way to the bottom. light floral scent with malt wafting though and a faint tingle on the nose suggesting hops content. tasted caramel and nut flavors in the initial sweet malt impression - toasty. bitterness creeps in soon after adding bite to the malt before crushing it with a heavy hops burst mid-palate. strong finish that is noticably more hops than malt (despite the touted &quot;balanced finish&quot;) and has great lingering power. felt smooth at first, then more medium-bodied and a bit chewier or oily. appropriate carbonation. very decent brew if you're in the mood for something less pale with plenty of hops.
Extracted serving type: unknown
True serving type: bottle
----------------------
Review 1170193:
this beer is meant to drink from the can - my quintessent

Artificial examples : 

In [ ]:
ex = "Had a can. Would love to try it on tap, and maybe in a bottle."
ex2 = "I had a bottle of beer and a can of coke"
ex3 = "Can I go to the bathroom I had a beer in a can even if I would have prefered it served by draft because I can."
ex4 = "Can I go to the bathroom I had a beer in a can even if I would have prefered it served by draft because I can. Next time, I will try a bottle."
ex_review = str("Find this one whenever you can.")
print(extract_serving_style(ex))
print(extract_serving_style(ex2))
print(extract_serving_style(ex3))
print(extract_serving_style(ex4))
print(differentiate_can(ex_review))

On peut supprimer ca ?   
Test on real reviews : 

In [ ]:
# DO NOT DELETE THIS CELL: used for testing below
df_nlp = df.sample(n=1000, random_state=46).copy(deep=True)

In [ ]:
t1 = time.time()
test = df_nlp.sample(15, random_state=46).copy(deep=True)
test['serving_style'] = test['text'].apply(lambda x: extract_serving_style(x))
t2 = time.time()
print('Time to run the function on 10 reviews :', round(t2 - t1, 2), 'seconds')

In [ ]:
# labeled by hand to check if the function works well
true_SS = ['bottle', 'bottle', None, 'draft', None, 'bottle', 'bottle', 'bottle', None, 'bottle', 'bottle', None, None, 'bottle', 'bottle']

In [ ]:
test['serving_style']

In [ ]:
print(f"Accuracy : {compute_accuracy(test['serving_style'], true_SS)*100}%")

### Countries grouping 🌍

In [ ]:
countries = pd.read_csv('data/countries.csv', usecols=['name', 'region', 'sub-region'])
countries.sample(1)

### Visualization 📊

In [ ]:
# print some reviews for each serving type
print('Reviews with bottle:\n')
for text in df[df['serving_type'] == 'bottle']['text'].sample(1):
    print(text)
    print('----------------------')
print('Reviews with can:\n')
for text in df[df['serving_type'] == 'can']['text'].sample(1):
    print(text)
    print('----------------------')
print('Reviews with draft:\n')
for text in df[df['serving_type'] == 'draft']['text'].sample(1):
    print(text)
    print('----------------------')

In [ ]:
# print the average value of the rating for each serving type
comp_str = 'rating'
print('Average ' + comp_str + ' for bottle: ', round(df[df['serving_type'] == 'bottle'][comp_str].mean(), 3))
print('Average ' + comp_str + ' for can: ', round(df[df['serving_type'] == 'can'][comp_str].mean(), 3))
print('Average ' + comp_str + ' for draft: ', round(df[df['serving_type'] == 'draft'][comp_str].mean(), 3))

In [ ]:
# show boxplot and histograms of ratings for bottle, can and draft
plt.figure(figsize=(10, 6))
plt.subplot(121)
sns.boxplot(data=df, hue='serving_type', y='rating')
plt.subplot(122)
sns.histplot(data=df, hue='serving_type', x='rating', kde=True, bins=20)
plt.tight_layout()
plt.show()

# Statistical Analysis 📈

### Measure the impact of each aspect on rating 📏

In [ ]:
# fit a linear regression line to the data for all 5 aspects
reg1 = LinearRegression().fit(df[['appearance']], df['rating'])
reg2 = LinearRegression().fit(df[['aroma']], df['rating'])
reg3 = LinearRegression().fit(df[['palate']], df['rating'])
reg4 = LinearRegression().fit(df[['taste']], df['rating'])
reg5 = LinearRegression().fit(df[['overall']], df['rating'])

# create subplots for all 5 aspects
fig, axs = plt.subplots(1, 5, figsize=(20, 5))

# plot the data and regression line for each aspect
sns.scatterplot(data=df, x='appearance', y='rating', s=0.5, ax=axs[0])
axs[0].plot(df[['appearance']], reg1.predict(df[['appearance']]), color='red')
axs[0].set_title('Appearance\nSlope = ' + str(round(reg1.coef_[0], 3)) + '\nR2 = ' + str(round(reg1.score(df[['appearance']], df['rating']), 3)))

sns.scatterplot(data=df, x='aroma', y='rating', s=0.5, ax=axs[1])
axs[1].plot(df[['aroma']], reg2.predict(df[['aroma']]), color='red')
axs[1].set_title('Aroma\nSlope = ' + str(round(reg2.coef_[0], 3)) + '\nR2 = ' + str(round(reg2.score(df[['aroma']], df['rating']), 3)))

sns.scatterplot(data=df, x='palate', y='rating', s=0.5, ax=axs[2])
axs[2].plot(df[['palate']], reg3.predict(df[['palate']]), color='red')
axs[2].set_title('Palate\nSlope = ' + str(round(reg3.coef_[0], 3)) + '\nR2 = ' + str(round(reg3.score(df[['palate']], df['rating']), 3)))

sns.scatterplot(data=df, x='taste', y='rating', s=0.5, ax=axs[3])
axs[3].plot(df[['taste']], reg4.predict(df[['taste']]), color='red')
axs[3].set_title('Taste\nSlope = ' + str(round(reg4.coef_[0], 3)) + '\nR2 = ' + str(round(reg4.score(df[['taste']], df['rating']), 3)))

sns.scatterplot(data=df, x='overall', y='rating', s=0.5, ax=axs[4])
axs[4].plot(df[['overall']], reg5.predict(df[['overall']]), color='red')
axs[4].set_title('Overall\nSlope = ' + str(round(reg5.coef_[0], 3)) + '\nR2 = ' + str(round(reg5.score(df[['overall']], df['rating']), 3)))

plt.tight_layout()
plt.show()

# should still add confidence intervals

It seems like Taste as the biggest impact on the rating while appearance has the least impact.

### Hypothesis testing on the new labels

In [ ]:
def do_ttest(type1, type2, category):
    ttest = ttest_ind(df[df['serving_type'] == type1][category],df[df['serving_type'] == type2][category])
    if ttest[1] > 0.05:
        print('The p-value is high (' + str(ttest[1]) + '), so we cannot reject the null hypothesis that the two samples have the same average')
    else:
        print('The p-value is very low (' + str(ttest[1]) + '), so we can reject the null hypothesis that the two samples have the same average')

In [ ]:
# define the columns to compare
columns = ['appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']

# create an empty matrix to store the p-values
p_values = np.zeros((len(columns), 3))

# perform the t-test and store the p-values in the matrix
for i in range(len(columns)):
    ttest1 = ttest_ind(df[df['serving_type'] == 'bottle'][columns[i]], df[df['serving_type'] == 'can'][columns[i]])
    ttest2 = ttest_ind(df[df['serving_type'] == 'can'][columns[i]], df[df['serving_type'] == 'draft'][columns[i]])
    ttest3 = ttest_ind(df[df['serving_type'] == 'draft'][columns[i]], df[df['serving_type'] == 'bottle'][columns[i]])
    p_values[i][0] = ttest1[1]
    p_values[i][1] = ttest2[1]
    p_values[i][2] = ttest3[1]

# create the grouped bar chart
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(columns))
width = 0.25
rects1 = ax.bar(x - width, p_values[:,0], width, label='Bottle vs Can')
rects2 = ax.bar(x, p_values[:,1], width, label='Can vs Draft')
rects3 = ax.bar(x + width, p_values[:,2], width, label='Draft vs Bottle')
ax.set_xticks(x)
# ax.set_yscale('log')
ax.set_xticklabels(columns)
ax.set_ylabel('p-value')
ax.set_title('Comparison of Aspects between Service Types')
ax.legend()
plt.show()

All the p-values are below 0.05 (expect 1), so we can reject the null hypothesis and conclude that there is a significant difference between the groups.

In [ ]:
# t-test for aroma betwwen draft and bottle
do_ttest('draft', 'bottle', 'aroma')

In [ ]:
# show a boxplot of aroma for draft and bottle
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[df['serving_type'].isin(['draft', 'bottle'])], x='serving_type', y='aroma')
plt.show()